In [ ]:
import os
save_directory= "Model"
train_address = "Train"
test_address = "Test"

if not os.path.exists(test_address):
  os.makedirs(test_address)
  !pip install gdown
  !pip install efficientnet
  !gdown https://drive.google.com/uc?id=1rxlpshNKP2gQ6NP9Wpvq-jUdbk5t6-Ub
  !unzip Cards.zip
  !mv Cards Test/Cards
  !gdown https://drive.google.com/uc?id=1lihZRRy1PSOw1npY70zn9U-CplagKOnA
  !unzip Center.zip
  !mv Center Test/Center
  !gdown https://drive.google.com/uc?id=1s6QlqMnLwZ749iEn0FkzVOU5IwiqwYWy
  !unzip Corner.zip
  !mv Corner Test/Corner
  !gdown https://drive.google.com/uc?id=15eA8iN0_aqtnqY66-aaTKlhPGWUL6SDz
  !unzip Free-Kick.zip
  !mv Free-Kick Test/Free-Kick
  !gdown https://drive.google.com/uc?id=1XKVo0Bim9RF1ktFbQhvpEt6SJhIZRzSn
  !unzip Left.zip
  !mv Left Test/Left
  !gdown https://drive.google.com/uc?id=1cMHgAw0YGJCEMuFYpO0cTiHQ8hdW-xtH
  !unzip Penalty.zip
  !mv Penalty Test/Penalty
  !gdown https://drive.google.com/uc?id=1lI3LfWtqD9PcOxkPaxgvMiriYtZ1oB36
  !unzip Right.zip
  !mv Right Test/Right
  !gdown https://drive.google.com/uc?id=14ZPdWb4Gufjl3wlc_ZA9XS_j_5z6gNct
  !unzip Tackle.zip
  !mv Tackle Test/Tackle
  !gdown https://drive.google.com/uc?id=1wwFom1He6S_arcYF9zpDL59vuq8b2DDS
  !unzip To-Subtitue.zip
  !mv To-Subtitue Test/To-Subtitue
if not os.path.exists(train_address):
  os.makedirs(train_address)
  !pip install gdown
  !gdown https://drive.google.com/uc?id=1sQuc6JoE5CoKq6JYe6i75AoWEBngAVb8
  !unzip Cards.zip
  !mv Cards Train/Cards
  !gdown https://drive.google.com/uc?id=1kUOQrJYUDnc9GZioxd3mMLuSsVoTG7Li
  !unzip Center.zip
  !mv Center Train/Center
  !gdown https://drive.google.com/uc?id=1PQeQOjBD5N728qVk5jkdXhwgeGc8nbIw
  !unzip Corner.zip
  !mv Corner Train/Corner
  !gdown https://drive.google.com/uc?id=1PSHANO0EuICFGuUJ6We8rHpUVMsL3Ecc
  !unzip Free-Kick.zip
  !mv Free-Kick Train/Free-Kick
  !gdown https://drive.google.com/uc?id=1pyVHfUI7CaYBlfuqY4BnGUe1t0QltHNU
  !unzip Left.zip
  !mv Left Train/Left
  !gdown https://drive.google.com/uc?id=1s5BPVAwbRSSmagK8z6PWvVNMFmKNkG8v
  !unzip Penalty.zip
  !mv Penalty Train/Penalty
  !gdown https://drive.google.com/uc?id=1hoJ1t3eQ2hlydn4GdCjYoNQpR9cmqqAi
  !unzip Right.zip
  !mv Right Train/Right
  !gdown https://drive.google.com/uc?id=1u0dx-RA_-m6e9AlkUDqzUaa9C3Rh1X5q
  !unzip Tackle.zip
  !mv Tackle Train/Tackle
  !gdown https://drive.google.com/uc?id=1zkOsebeFWHD72A08xO-pYlubKguWrudB
  !unzip To-Subtitue.zip
  !mv "To Subtitue" Train/To-Subtitue

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Lambda
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.optimizers import Adam,RMSprop, SGD , Nadam, Adadelta
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2 
from keras.backend import sigmoid
from keras.utils.generic_utils import get_custom_objects
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from imutils import paths
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import pandas as pd
import numpy as np
import random
import pickle
import cv2
import efficientnet.keras as enet
matplotlib.use("Agg")
import os
from keras.layers import GlobalAveragePooling2D
from keras.layers import Input, Dense, Flatten, Multiply, add

In [ ]:
!rm "Train/To-Subtitue/To Subtitue__21__14.jpg"
print("[INFO] loading train images...")
imagePaths = sorted(list(paths.list_images(train_address)))
random.seed(42)
random.shuffle(imagePaths)
print(len(imagePaths))

In [ ]:
EPOCHS = 20
INIT_LR = 0.0001
BS = 16
IMAGE_DIMS = (112,112, 3)
data = []
categoryLabels = []

cnt = 0;
for imagePath in imagePaths:
        if (cnt % 500) == 0 :
            print(cnt)
        image = cv2.imread(imagePath)
        image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = img_to_array(image)
        data.append(image)
        (cat, _,_) = imagePath.split(os.path.sep)[-1].split("__")
        categoryLabels.append(cat)
        cnt = cnt + 1;



print("[INFO] loading test images...")
imagePaths2 = sorted(list(paths.list_images(test_address)))
random.seed(42)
random.shuffle(imagePaths2)
cnt = 0;
for imagePath in imagePaths2:
        if (cnt % 500) == 0 :
            print(cnt)
        image = cv2.imread(imagePath)
        image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = img_to_array(image)
        data.append(image)
        (cat, _,_) = imagePath.split(os.path.sep)[-1].split("__")
        categoryLabels.append(cat)
        cnt = cnt + 1;


In [ ]:
print(len(data))

In [ ]:

import keras 
class SwishActivation(Activation):
    
    def __init__(self, activation, **kwargs):
        super(SwishActivation, self).__init__(activation, **kwargs)
        self.__name__ = 'swish_act'

def swish_act(x, beta = 1):
    return (x * sigmoid(beta * x))


get_custom_objects().update({'swish_act': SwishActivation(swish_act)})


class SoccerNet:

	@staticmethod
	def build_category_branch(inputs, numCategories,
		finalAct="softmax", chanDim=-1):
		x = inputs.output

		x = BatchNormalization(axis=chanDim)(x)
		x = Dropout(0.7)(x)

		x = Dense(512)(x)
		x = BatchNormalization(axis=chanDim)(x)
		x = Activation(swish_act)(x)
		x = Dropout(0.5)(x)

		x = Dense(512)(x)
		x = BatchNormalization(axis=chanDim)(x)
		x = Activation(swish_act)(x)
		x = Dense(numCategories)(x)
		x = Activation(finalAct, name="category_output")(x)
		return x



	@staticmethod
	def build(width, height, numCategories,
		finalAct="softmax"):
		inputShape = (height, width, 3)
		chanDim = -1
		inputs = Input(shape=inputShape)
		print("nputshape",inputShape)
		model = keras.applications.MobileNetV2(include_top=False,input_tensor=inputs, pooling='avg', weights='imagenet')
		categoryBranch = SoccerNet.build_category_branch(model,
			numCategories, finalAct=finalAct, chanDim=chanDim)
		model = Model(
			inputs=inputs,
			outputs=[categoryBranch],
			name="soccerNet")
		return model

In [ ]:

data = np.array(data, dtype="float") / 255.0
print("[INFO] data matrix: {} images ({:.2f}MB)".format(
	len(imagePaths), data.nbytes / (1024 * 1000.0)))

categoryLabels = np.array(categoryLabels)
print(np.unique(categoryLabels))
print("[INFO] binarizing labels...")
categoryLB = LabelBinarizer()
categoryLabels = categoryLB.fit_transform(categoryLabels)


split = train_test_split(data, categoryLabels,
	test_size=0.833,shuffle = False)  
(trainX, testX, trainCategoryY, testCategoryY) = split

model = SoccerNet.build(IMAGE_DIMS[0], IMAGE_DIMS[1],
	numCategories=len(categoryLB.classes_),
	finalAct="softmax")


print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(optimizer=opt, loss="categorical_crossentropy",
	metrics=["accuracy"])

if not os.path.exists(save_directory):
    os.mkdir(save_directory)
    print("Directory " , save_directory ,  " Created ")
H = model.fit(x=trainX,y=trainCategoryY,
	validation_data=(testX,testCategoryY),
	epochs=EPOCHS,
  batch_size=BS, 
	verbose=1
  )

In [ ]:
y_pred = model.predict(testX)


y_true_category = np.argmax(testCategoryY, axis=1)
y_pred_category = np.argmax(y_pred, axis=1) 



f = open("{}/Report_Performance.txt".format(save_directory), "w")
accuracy_category= "\n accuracy : " + str(accuracy_score(y_true_category, y_pred_category))
print("accuracy : " , accuracy_category)
f.write(accuracy_category)
from sklearn.metrics import f1_score
f1_score = f1_score(y_true_category, y_pred_category, average='macro')
print("f1_score : " , f1_score)
f1_score = "f1_score : " + str(f1_score)
f.write(f1_score)
from sklearn.metrics import recall_score
recall_score = recall_score(y_true_category, y_pred_category, average='macro')
print("recall_score : " , recall_score)
recall_score = "recall_score : " + str(recall_score)
f.write(recall_score)
from sklearn.metrics import precision_score
precision_score = precision_score(y_true_category, y_pred_category, average='macro')
print("precision_score : " , precision_score)
precision_score = "precision_score : " + str(precision_score)
f.write(precision_score)


import json
history_dict = H.history
json.dump(history_dict, open('{}/file.json'.format(save_directory), 'w'))


categoryLB2 = np.unique(categoryLB.inverse_transform(testCategoryY))


plt.figure(figsize=(20,20))
clf_report = classification_report(y_true_category,
                                   y_pred_category,
                                   target_names=categoryLB2,
                                   output_dict=True)
                                   
ax = sns.heatmap(pd.DataFrame(clf_report).iloc[:-1, :].T, annot=True,cmap="binary" )
ax.set_title('Classification Report - Category');
plt.savefig('{}/Classification_Report_Category.png'.format(save_directory), dpi=200, format='png', bbox_inches='tight')                                    
                          

lossNames = ["loss"]
plt.style.use("ggplot")
(fig, ax) = plt.subplots(3, 1, figsize=(13, 13))
for (i, l) in enumerate(lossNames):

	title = "Loss for {}".format(l) if l != "loss" else "Total loss"
	ax[i].set_title(title)
	ax[i].set_xlabel("Epoch #")
	ax[i].set_ylabel("Loss")
	ax[i].plot(np.arange(0, EPOCHS), H.history[l], label=l)
	ax[i].plot(np.arange(0, EPOCHS), H.history["val_" + l],
		label="val_" + l)
	ax[i].legend()

plt.tight_layout()
plt.savefig("{}/losses.png".format(save_directory))
plt.close()


accuracyNames = ["accuracy"]
plt.style.use("ggplot")
(fig, ax) = plt.subplots(2, 1, figsize=(8, 8))
for (i, l) in enumerate(accuracyNames):

	ax[i].set_title("Accuracy for {}".format(l))
	ax[i].set_xlabel("Epoch #")
	ax[i].set_ylabel("Accuracy")
	ax[i].plot(np.arange(0, EPOCHS), H.history[l], label=l)
	ax[i].plot(np.arange(0, EPOCHS), H.history["val_" + l],
		label="val_" + l)
	ax[i].legend()

plt.tight_layout()
plt.savefig("{}/accs.png".format(save_directory))
plt.close()


print("[INFO] serializing category label binarizer...")
f = open("{}/category_lb.pickle".format(save_directory), "wb")
f.write(pickle.dumps(categoryLB))
f.close()

print("[INFO] serializing network...")
model.save("{}/model_saved".format(save_directory))

full_model = tf.function(lambda x: model(x)) 
full_model = full_model.get_concrete_function( 
tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype))
frozen_func = convert_variables_to_constants_v2(full_model) 
frozen_func.graph.as_graph_def() 
layers = [op.name for op in frozen_func.graph.get_operations()]

'''
tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
                  logdir="./{}".format(save_directory),
                  name="frozen_graph.pb",          
                  as_text=False)'''
'''
tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
                  logdir="./{}".format(save_directory),
                  name="frozen_graph.pbtxt",          
                  as_text=True)
'''

plt.figure(figsize=(20,20))
ax = sns.heatmap(confusion_matrix(y_true_category,y_pred_category), cmap="binary",annot=True,fmt="d")
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix - Category'); 
ax.set_xticklabels(categoryLB2, rotation=90)
ax.set_yticklabels(categoryLB2, rotation=0) 
plt.savefig("{}/Confusion_Matrix_Category.jpg".format(save_directory))



In [ ]:
!zip -r "MobileNetV2.zip" Model